In [ ]:
# | default_exp transforms/spatial

# Imports

In [ ]:
# | export


import torch
from monai.data import MetaTensor
from monai.transforms.spatial.dictionary import RandFlipd, Resized

# Transforms

In [ ]:
# | export


class RandFlipWithCropTrackingd(RandFlipd):
    def __init__(
        self,
        crop_offset_key: str = "crop_offset",
        original_shape_key: str = "original_shape",
        *args,
        **kwargs,
    ) -> MetaTensor:
        super().__init__(*args, **kwargs)
        self.crop_offset_key = crop_offset_key
        self.original_shape_key = original_shape_key

    def __call__(self, data, *args, **kwargs):
        output = super().__call__(data, *args, **kwargs)
        # Check if image was flipped and was cropped previously
        if self._do_transform and self.crop_offset_key in data:
            # Ensure all inputs had input of same size for consistent crop value
            shapes = [data[key].shape for key in self.keys]
            if not all(shape == shapes[0] for shape in shapes):
                raise ValueError("All inputs must have the same shape for consistent crop value")
            crop_shape = shapes[0]
            orig_shape = torch.Tensor(data[self.original_shape_key])
            # New crop value will be flipped
            crop_offset = torch.Tensor(data[self.crop_offset_key])
            # Flip crop value
            crop_offset[-1] = orig_shape[-1] - crop_offset[-1] - crop_shape[-1]
            # Update crop value
            output[self.crop_offset_key] = crop_offset
        return output

In [ ]:
img = torch.rand(4, 100, 100, 100)
x = {"image": img, "crop_offset": (10, 20, 30), "original_shape": (200, 200, 200)}

RandFlipWithCropTrackingd(keys=["image"], prob=1.0)(x)


{
    'image': metatensor([[[[1.2556e-01, 9.4766e-01, 7.2156e-01,  ..., 1.8215e-01,
           1.3025e-01, 6.1415e-01],
          [9.9686e-01, 4.6192e-02, 9.0233e-01,  ..., 7.5272e-02,
           3.5425e-01, 1.0359e-01],
          [2.7953e-01, 4.0598e-01, 7.5143e-01,  ..., 3.9831e-01,
           2.6774e-01, 8.4779e-01],
          ...,
          [3.5447e-01, 7.2037e-02, 3.7588e-01,  ..., 7.2441e-01,
           7.0234e-02, 5.2725e-01],
          [5.7474e-01, 7.7326e-01, 5.4428e-02,  ..., 8.0043e-01,
           3.7068e-01, 2.2962e-01],
          [9.2324e-01, 8.0576e-01, 1.1867e-01,  ..., 3.0535e-01,
           7.8999e-01, 5.3779e-01]],

         [[2.3683e-01, 8.6547e-01, 5.4783e-01,  ..., 8.2040e-01,
           7.1317e-01, 2.1071e-01],
          [2.0908e-01, 8.6550e-01, 7.5535e-02,  ..., 9.1400e-01,
           1.2350e-01, 3.2234e-01],
          [6.7998e-01, 9.1790e-01, 5.8289e-01,  ..., 4.5103e-01,
           8.2184e-01, 7.1126e-01],
          ...,
          [1.7351e-01, 9.4925e-01, 8.34

In [ ]:
# | export


class ResizedWithCropTrackingd(Resized):
    def __init__(
        self,
        crop_offset_key: str = "crop_offset",
        original_shape_key: str = "original_shape",
        *args,
        **kwargs,
    ) -> MetaTensor:
        super().__init__(*args, **kwargs)
        self.crop_offset_key = crop_offset_key
        self.original_shape_key = original_shape_key

    def __call__(self, data, *args, **kwargs):
        # Check if image was cropped previously
        if self.crop_offset_key in data:
            # Ensure all inputs had input of same size for consistent crop value
            shapes = [data[key].shape for key in self.keys]
            if not all(shape == shapes[0] for shape in shapes):
                raise ValueError("All inputs must have the same shape for consistent crop value")
            old_shape = data[self.keys[0]].shape[1:]

        output = super().__call__(data, *args, **kwargs)

        # Check if image was cropped previously
        if self.crop_offset_key in data:
            # Calculate scale of resizing along every axis
            new_shape = output[self.keys[0]].shape[1:]
            scale = torch.tensor(new_shape) / torch.tensor(old_shape)

            # Update crop_offset
            crop_offset = torch.Tensor(data[self.crop_offset_key])
            output[self.crop_offset_key] = (crop_offset * scale).int()

            # Update original_shape
            if self.original_shape_key in data:
                original_shape = torch.Tensor(data[self.original_shape_key])
                output[self.original_shape_key] = (original_shape * scale).int()

        return output

In [ ]:
img = torch.rand(4, 100, 100, 100)
x = {"image": img, "crop_offset": (10, 20, 30), "original_shape": (200, 200, 200)}

ResizedWithCropTrackingd(keys=["image"], spatial_size=(60, 60, 60))(x)


{
    'image': metatensor([[[[0.2471, 0.4995, 0.5993,  ..., 0.6837, 0.4922, 0.3836],
          [0.4113, 0.4792, 0.4810,  ..., 0.5323, 0.5230, 0.4629],
          [0.6377, 0.6658, 0.4599,  ..., 0.5872, 0.6815, 0.5865],
          ...,
          [0.5383, 0.4991, 0.4617,  ..., 0.5246, 0.5131, 0.5159],
          [0.5867, 0.5324, 0.5722,  ..., 0.4862, 0.4494, 0.6092],
          [0.5035, 0.4671, 0.5119,  ..., 0.4933, 0.3926, 0.5348]],

         [[0.3539, 0.4561, 0.6127,  ..., 0.5380, 0.3610, 0.3665],
          [0.4356, 0.4769, 0.5482,  ..., 0.5419, 0.4622, 0.5637],
          [0.5579, 0.6443, 0.3892,  ..., 0.5879, 0.6259, 0.5324],
          ...,
          [0.5120, 0.4244, 0.4199,  ..., 0.5327, 0.4986, 0.5241],
          [0.5663, 0.4930, 0.4757,  ..., 0.5938, 0.5108, 0.4215],
          [0.4744, 0.3861, 0.3822,  ..., 0.5277, 0.5098, 0.4552]],

         [[0.5599, 0.4154, 0.4758,  ..., 0.5375, 0.5226, 0.5269],
          [0.6411, 0.5608, 0.4648,  ..., 0.5323, 0.5107, 0.6235],
          [0.6394, 0.6

# nbdev

In [ ]:
!nbdev_export